<a href="https://colab.research.google.com/github/gc2321/LangChain-Python/blob/main/2_1_doc_loaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
!pip install --upgrade --quiet langchain-openai langchain-community
!pip install -quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00

Usage:   
  pip3 install [op

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
open_ai_api ="/content/gdrive/MyDrive/llm/api_key.txt"
f = open(open_ai_api)
api_key = f.read()

# History Quuiz

Use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain_openai import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI


In [ ]:
class HistoryQuiz():

    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:

            "On what date did World War 2 end?"

        '''
        # System context:
        system_template="Write single quiz questions about {topic}. Return the quiz question."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # human request
        human_template="{question_request}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # compile prompt
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        # add variables
        request = chat_prompt.format_prompt(topic=topic,question_request="Give me a quiz question where the correct answer is a specific date.").to_messages()

        # send request
        chat = ChatOpenAI(openai_api_key=api_key)
        result = chat.invoke(request)

        return result.content

    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        # Datetime Parser
        output_parser = DatetimeOutputParser()

        # SYSTEM Template
        system_template = "Answer quiz questions with just a date."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)


        # HUMAN Template
        human_template = """Answer the user's question:

        {question}

        {format_instructions}"""
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # Compile ChatTemplate
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

        # Insert question and format instructions

        request = chat_prompt.format_prompt(question=question,
                                            format_instructions=output_parser.get_format_instructions()).to_messages()



        # Chat Bot
        chat = ChatOpenAI(openai_api_key=api_key)
        result = chat.invoke(request)
        # Format Request to datetime
        correct_datetime = output_parser.parse(result.content)
        return correct_datetime

    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)

        # Get the year, month, and day from the user
        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))

        # Create a datetime object
        user_datetime = datetime(year, month, day)

        return user_datetime


    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''

        # Calculate the difference between the dates
        difference = user_answer - ai_answer

        # Format the difference into a string
        formatted_difference = str(difference)

        # Return the string reporting the difference
        print("The difference between the dates is:", formatted_difference)



In [ ]:
quiz_bot = HistoryQuiz()

In [ ]:
question = quiz_bot.create_history_question(topic='World War 2')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
question

'On what date did the attack on Pearl Harbor take place, drawing the United States into World War II?'

In [ ]:
ai_answer = quiz_bot.get_AI_answer(question)

In [ ]:
ai_answer

datetime.datetime(1941, 12, 7, 7, 55)

In [ ]:
user_answer = quiz_bot.get_user_answer(question)

On what date did the attack on Pearl Harbor take place, drawing the United States into World War II?
Enter the year: 1942
Enter the month (1-12): 12
Enter the day (1-31): 12


In [ ]:
user_answer

datetime.datetime(1942, 12, 12, 0, 0)

In [ ]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: 369 days, 16:05:00
